In [ ]:
#importing libraries and packages
import pandas as pd
import numpy as np
from datetime import datetime

from scipy import optimize
from scipy import integrate
import random

#for plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# for dashboard
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

# set parameter for plotting 
mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 200)

In [ ]:
# import local file to create dataframe and set date in ascending order
df_analyse=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';')  
df_analyse.sort_values('date',ascending=True).head()

# Simulative approach to calculate SIR curves

### To describe the spread of infectious diseases, mediacal researchers and mathematicians have developed a sophisticated mathematical model - SIR Model 

- S = Succeptible to becoming infected
- I = Infected through contact with someone already infected
- R = Recovered, no longer sick or infected

Detailed theory and application can be found here; http://www.pandemsim.com/data/index.php/make-your-own-sir-model/

In [ ]:
# set some basic parameters
# beta/gamma is denoted as  'basic reproduction number'

N0=1000000 #max susceptible population
beta=0.4   # infection spread dynamics
gamma=0.1  # recovery rate


# condition I0+S0+R0=N0
I0=df_analyse.Germany[35]
S0=N0-I0
R0=0

In [ ]:
# defining SIR model function

def SIR_model(SIR,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return([dS_dt,dI_dt,dR_dt])

In [ ]:
SIR=np.array([S0,I0,R0])
propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recoverd':R0})



for each_t in np.arange(100):
   
    new_delta_vec=SIR_model(SIR,beta,gamma)
   
    SIR=SIR+new_delta_vec
    
    propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True)

In [ ]:
fig, ax1 = plt.subplots(1, 1)

ax1.plot(propagation_rates.index,propagation_rates.infected,label='infected',color='k')
ax1.plot(propagation_rates.index,propagation_rates.recovered,label='recovered')
ax1.plot(propagation_rates.index,propagation_rates.susceptible,label='susceptible')

ax1.set_ylim(10, 1000000)
ax1.set_yscale('linear')
ax1.set_title('Szenario SIR simulations  (demonstration purposes only)',size=16)
ax1.set_xlabel('time in days',size=16)
ax1.legend(loc='best',
           prop={'size': 16});

# Calculating Fitting Parameters of SIR Curve

In [ ]:
# Skipped the data with 0 values
ydata = np.array(df_analyse.Germany[35:])
t=np.arange(len(ydata))

In [ ]:
# ensure re-initialization 
I0=ydata[0]
S0=N0-I0
R0=0

In [ ]:
# with respect to previous SIR model function, this function adds independent variable time t (days)
def SIR_model_t(SIR,t,beta,gamma):
    ''' Simple SIR model
        S: susceptible population
        t: time step, mandatory for integral.odeint
        I: infected people
        R: recovered people
        beta: 
        
        overall condition is that the sum of changes (differnces) sum up to 0
        dS+dI+dR=0
        S+I+R= N (constant size of population)
    
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0          #S*I is the 
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    return dS_dt,dI_dt,dR_dt

In [ ]:
def fit_odeint(x, beta, gamma):
    '''
    helper function for the integration
    '''
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1] # we only would like to get dI

In [ ]:
# example curve of our differential equationa
popt=[0.4,0.1]
fit_odeint(t, *popt)

In [ ]:
# the resulting curve has to be fitted
# free parameters are here beta and gamma

popt, pcov = optimize.curve_fit(fit_odeint, t, ydata)
perr = np.sqrt(np.diag(pcov))
    
print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])

In [ ]:
# get the final fitted curve
fitted=fit_odeint(t, *popt)

In [ ]:
# plotting original and fitted data for Germany
plt.semilogy(t, ydata, 'o')
plt.semilogy(t, fitted)
plt.title("Fitted SIR model for Germany")
plt.ylabel("Number of Infected Population")
plt.xlabel("Timeline (Days)")
plt.show()
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
print("Basic Reproduction Number R0 " , popt[0]/ popt[1])
print("This ratio is derived as the expected number of new infections (these new infections are sometimes called secondary infections from a single infection in a population where all subjects are susceptible. @wiki")

# Dynamic beta in SIR (infection rate)

In [ ]:
t_initial=28 # When infection started rising
t_intro_measures=14 # When lockdown was implemented
t_hold=21 # Lockdown held for the timeperiod
t_relax=21 # Ease in lockdown measures introduced

beta_max=0.4
beta_min=0.11
gamma=0.1

# concatinating the parameters with respected time period

pd_beta=np.concatenate((np.array(t_initial*[beta_max]),
                       np.linspace(beta_max,beta_min,t_intro_measures),
                       np.array(t_hold*[beta_min]),
                        np.linspace(beta_min,beta_max,t_relax),
                       ))

In [ ]:
# check above setted parameter for relatable time period
pd_beta

In [ ]:
# SIR modelling
SIR=np.array([S0,I0,R0])
propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recoverd':R0})



for each_beta in pd_beta:
   
    new_delta_vec=SIR_model(SIR,each_beta,gamma)
   
    SIR=SIR+new_delta_vec
    
    propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True)

In [ ]:
fig, ax1 = plt.subplots(1, 1)

ax1.plot(propagation_rates.index,propagation_rates.infected,label='infected',linewidth=3)

t_phases=np.array([t_initial,t_intro_measures,t_hold,t_relax]).cumsum()
ax1.bar(np.arange(len(ydata)),ydata, width=0.8,label=' current infected Germany',color='r')
ax1.axvspan(0,t_phases[0], facecolor='b', alpha=0.2,label='no measures')
ax1.axvspan(t_phases[0],t_phases[1], facecolor='b', alpha=0.3,label='hard measures introduced')
ax1.axvspan(t_phases[1],t_phases[2], facecolor='b', alpha=0.4,label='hold measures')
ax1.axvspan(t_phases[2],t_phases[3], facecolor='b', alpha=0.5,label='relax measures')
ax1.axvspan(t_phases[3],len(propagation_rates.infected), facecolor='b', alpha=0.6,label='repead hard measures')

ax1.set_ylim(10, 1.5*max(propagation_rates.infected))
ax1.set_yscale('log')
ax1.set_title('Szenario SIR simulations  (demonstration purposes only)',size=16)
ax1.set_xlabel('time in days',size=16)
ax1.legend(loc='best',
           prop={'size': 16});

# Visual Dash Board for SIR Model

## 1. Data Preparation

In [ ]:
#importing data
df_raw = pd.read_csv('../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
country_list = df_raw['Country/Region'].unique()
date = df_raw.columns[4:]
df_final = pd.DataFrame({'Date': date})

#transforming the raw data to accessible for SIR model
for each in country_list:
    df_final[each] = np.array(df_raw[df_raw['Country/Region'] == each].iloc[:,4::].sum(axis=0)).T
    
df_final.to_csv("../data/raw/COVID-19/csse_covid_19_data/SIR_modelling.csv", sep = ';', index=False)

df_analyse=pd.read_csv('../data/raw/COVID-19/csse_covid_19_data/SIR_modelling.csv',sep=';')

df_analyse.sort_values('Date',ascending=True).head()

## 2. SIR model and Fitting of parameters

In [ ]:
# Intialize parameter
N0 = 1000000
beta = 0.4
gamma = 0.1
I0=df_analyse.Germany[35]
S0=N0-I0
R0=0

df_data = df_analyse[35:]
t = np.arange(df_data.shape[0])

#calculating optimize parameters for every country
for country in df_data.columns[1:]:
    
        # preparing ydata so that, SIR modellig starts when the cases are greater than 0, for every country
        
        ydata = np.array(df_data[df_data[country]>0][country]) 
        t = np.arange(len(ydata))
        I0=ydata[0]
        S0=N0-I0
        R0=0
        popt=[0.4,0.1]
        fit_odeint(t, *popt)
        popt, pcov = optimize.curve_fit(fit_odeint, t, ydata, maxfev=5000)
        perr = np.sqrt(np.diag(pcov))
        fitted=fit_odeint(t, *popt)
        fit_pad = np.concatenate((np.zeros(df_data.shape[0]-len(fitted)) ,fitted))
        df_data[country + '_fitted'] = fit_pad

df_data = df_data.reset_index(drop=True)
df_data.to_csv('../data/processed/SIR_fitted.csv', sep = ';')

In [ ]:
#creating plot for india to see that our calculation works
fig = go.Figure()
fig.add_trace(go.Scatter(x = df_data['Date'],y = df_data['US_fitted'],name= 'Fitted',
                             mode='markers+lines',line_width = 1,marker_size = 3),
             )

fig.add_trace(go.Scatter(x = df_data['Date'],y = df_data['US'],name= 'From_Source',
                             mode='markers+lines',line_width = 1,marker_size = 3),
                 )

fig.update_layout(title={'text': 'SIR fitted curve with confirmed cases for US','y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'},
                  xaxis_title='Timeline (Days)', yaxis_title='Total number of infected people',width=1000, height=800)
fig.update_yaxes(type = 'log')
fig.update_layout(xaxis_rangeslider_visible=True)

In [ ]:
# differentiating the presentation of curves for each country via unique color
color_list = []
for i in range(200):
    var = '#%02x%02x%02x'%(random.randint(0,255),random.randint(0,255),random.randint(0,255))
    color_list.append(var)

In [ ]:
# creating dashboard for SIR curve and source data for all countries 
fig = go.Figure()
app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data - Dash Board 2
    
    - The default layout
        * Y-axis shows the confirmed infected cases in log-scale
        * X-axis shows the timeline in days
    
    - With the dropdown menu, user can visualize the curves for multiple countries

    - There are two plots in the dash board for each country:
    
    1. Plot of confirmed infected cases with respect to timeline.
    2. Plot of SIR curve respect to timeline. 
    
    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),
    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_data.columns[1:200]],
        value=['Germany','India'], # which are pre-selected
        multi=True),dcc.Graph(figure=fig, id='main_window_slope')])

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure(country_list):
    
    my_yaxis={'type':"log",'title':'Confirmed number of infected people (From johns hopkins csse, log-scale)'}
    traces = []
    x = 0
    for each in country_list:
        traces.append(dict(x=df_data['Date'],y=df_data[each],
                                mode='line', line = dict(color = color_list[x]), opacity=1.0,name=each))
        traces.append(dict(x=df_data['Date'],
                                y=df_data[each+'_fitted'],
                                mode='markers+lines',line = dict(color=color_list[x]), opacity=1.0,name=each+'_simulated'))

        x = x+1
    return {
            'data': traces,
            'layout': dict (
                width=1280,height=720,
                xaxis={'title':'Timeline(days)','tickangle':-45,'nticks':20,
                'tickfont':dict(size=14,color="#0c6887"),},yaxis=my_yaxis)}


if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)